In [1]:
!docker version

Client:
 Version:      1.13.1
 API version:  1.26
 Go version:   go1.6.2
 Git commit:   092cba3
 Built:        Thu Nov  2 20:40:23 2017
 OS/Arch:      linux/amd64

Server:
 Version:      1.13.1
 API version:  1.26 (minimum version 1.12)
 Go version:   go1.6.2
 Git commit:   092cba3
 Built:        Thu Nov  2 20:40:23 2017
 OS/Arch:      linux/amd64
 Experimental: false


In [2]:
!mkdir flaskwebapp
!mkdir flaskwebapp/code
!mkdir flaskwebapp/etc

mkdir: cannot create directory ‘flaskwebapp’: File exists
mkdir: cannot create directory ‘flaskwebapp/code’: File exists
mkdir: cannot create directory ‘flaskwebapp/etc’: File exists


In [3]:
!cp /poc/AZ/Kate/returnsModel_2.1-8.tar.gz flaskwebapp/code/
!cp /poc/AZ/Kate/OR_IR6_2017-11-28_modelling.RData flaskwebapp/code/

In [4]:
import os
from os import path
import json 

image_name = "thbeh/collections"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'Dockerfile')

In [5]:
%%writefile flaskwebapp/packages.R
#install.packages(c("randomForest"), 
#                   repos = "http://cran.us.r-project.org", dependencies = TRUE)

## install.packages("/poc/AZ/Kate/returnsModel_2.1-6.tar.gz", repos = NULL, type = "source")


Overwriting flaskwebapp/packages.R


In [6]:
%%writefile flaskwebapp/code/scoreDrill.R

#suppressMessages(library(psych))
#suppressMessages(library(jsonlite))
#suppressMessages(library(sergeant))
suppressMessages(library(returnsModel))

options(warn = -1)

#* @post /predict
runModel <- function(nr) {
    RunModellingAndScoringTest(model="OR", returnName="IR6", projectStage="TEST", nrows=nr)
}

#* @post /domodelling
refreshmodel <- function(model="OR", returnName="IR6", projectStage="TEST", nrows=1000){
    
    RefreshModel(model=model, returnName=returnName, projectStage=projectStage, nrows=nrows)
    
    #output <- RefreshModel(model=model, returnName=returnName, projectStage=projectStage, nrows=nrows)
    #AssignOutput(output, env= environment())
  
    #runDate <- format(Sys.Date(), "%Y-%m-%d")
    #imageName <- paste0("/code/", model,"_", returnName, "_", runDate, "_modelling.RData") 
    #save(list=names(output), file=imageName)
    #imageName
}


#* @post /testload
testload <- function(){
    e <<- .GlobalEnv
    load("/code/OR_IR6_2017-11-28_modelling.RData", envir=e)
    print(e$covYearsBack)
}


#* @post /doscoring
doscoring <- function(model="OR", returnName="IR6", modelRunDate="2017-11-28", projectStage="TEST", nrows=1000, 
              irdNumbersForScoring=""){
    e <<- .GlobalEnv
    imageName <- paste0(model, "_", returnName, "_", modelRunDate, "_modelling.RData")
    load(paste0("/code/", imageName), envir=e)
    
    e$model <- model
    e$returnName <- returnName
    e$modelRunDate <- modelRunDate
    e$projectStage <- projectStage
    e$nrows <- nrows
    e$irdNumbersForScoring <- irdNumbersForScoring
    
    DoScoring(model=e$model, returnName=e$returnName, modelRunDate=e$modelRunDate, projectStage=e$projectStage, 
              nrows=e$nrows, irdNumbersForScoring=e$irdNumbersForScoring, e=e)
    
}


Overwriting flaskwebapp/code/scoreDrill.R


In [7]:
%%writefile flaskwebapp/etc/supervisord.conf
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:plumber]
command=/usr/bin/Rscript -e "pr <- plumber::plumb('/code/scoreDrill.R'); pr$run(host='0.0.0.0', port=8000)"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Overwriting flaskwebapp/etc/supervisord.conf


In [8]:
%%writefile flaskwebapp/Dockerfile
FROM thbeh/r-base:3.4.2

RUN apt-get update && apt-get install -y supervisor libxml2-dev

COPY packages.R /tmp/
COPY scoreDrill.R /tmp/

RUN mkdir /code
ADD code /code
ADD etc /etc

RUN Rscript /tmp/packages.R
RUN R CMD INSTALL /code/returnsModel_2.1-8.tar.gz

EXPOSE 8000
CMD ["supervisord","-c", "/etc/supervisord.conf"]


Overwriting flaskwebapp/Dockerfile


In [9]:
! ls -l flaskwebapp/code

total 255899
-rwxrwxrwx 1 user00 user00       885 Nov 26 20:36 kill_supervisor.py
-rw-r--r-- 1 eked67 eked67 261555158 Nov 29 05:00 OR_IR6_2017-11-28_modelling.RData
-rw-rw-rw- 1 user00 user00    159017 Nov 27 03:50 returnsModel_2.1-6.tar.gz
-rw-rw-rw- 1 user00 user00    160068 Nov 28 02:35 returnsModel_2.1-7.tar.gz
-rw-r--r-- 1 eked67 eked67    161426 Nov 29 05:00 returnsModel_2.1-8.tar.gz
-rwxrwxrwx 1 user00 user00       377 Nov 27 05:47 runModel.R
-rw-rw-rw- 1 user00 user00      1767 Nov 29 05:00 scoreDrill.R


In [10]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon 262.1 MB
Step 1/11 : FROM thbeh/r-base:3.4.2
 ---> e86845d98544
Step 2/11 : RUN apt-get update && apt-get install -y supervisor libxml2-dev
 ---> Running in 9749bb4ac61b
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Ign:3 http://package.mapr.com/releases/v5.2.2/ubuntu binary InRelease
Get:4 http://ppa.launchpad.net/marutter/c2d4u/ubuntu xenial InRelease [17.5 kB]
Ign:5 http://package.mapr.com/releases/MEP/MEP-3.0.1/ubuntu binary InRelease
Get:6 http://package.mapr.com/releases/v5.2.2/ubuntu binary Release [2,728 B]
Get:7 http://package.mapr.com/releases/MEP/MEP-3.0.1/ubuntu binary Release [2,728 B]
Get:8 http://package.mapr.com/releases/v5.2.2/ubuntu binary Release.gpg [475 B]
Get:9 http://package.mapr.com/releases/MEP/MEP-3.0.1/ubuntu binary Release.gpg [475 B]
Get:10 http://package.mapr.com/releases/v5.2.2/ubuntu binary/trusty amd64 Packages [3,249 B]
G

In [ ]:
!docker rm domodelling

domodelling


In [ ]:
!docker run -it -e MAPR_CLUSTER=maprlab -e MAPR_CLDB_HOSTS=mapr-1,mapr-2,mapr-3 -e MAPR_CONTAINER_USER=mapr -e MAPR_CONTAINER_UID=5000 -e MAPR_CONTAINER_GID=5000 -e MAPR_CONTAINER_GROUP=mapr -e MAPR_MOUNT_POINT=/mapr --device /dev/fuse --security-opt apparmor:unconfined --name doscoring -p 6002:8000 $image_name 

[!p>
Testing for cluster user account... 

(BEnter MapR cluster user name: mapr
 
User 'mapr' does not exist. Creating new cluster user account... 

Enter 'mapr' uid: 5000
 Enter 'mapr' group name: mapr
 Enter 'mapr' gid: 5000
 Enter 'mapr' password: mapr
 ...Success 

Configuring MapR client ( -c -C mapr-1,mapr-2,mapr-3 -N maprlab)... 

create /opt/mapr/conf/conf.old
Configuring Hadoop-2.7.0 at /opt/mapr/hadoop/hadoop-2.7.0
Done configuring Hadoop
CLDB node list: mapr-1:7222,mapr-2:7222,mapr-3:7222
Zookeeper node list: 

...Success 

2017-11-29 05:02:09,700 INFO supervisord started with pid 944
2017-11-29 05:02:10,703 INFO spawned: 'program_exit' with pid 947
2017-11-29 05:02:10,705 INFO spawned: 'plumber' with pid 948
2017-11-29 05:02:12,121 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2017-11-29 05:02:16,126 INFO success: plumber entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
